In [ ]:
%load_ext autoreload
%autoreload 2

import json
import os
import cv2
from matplotlib import pyplot as plt
import pandas as pd
from research.utils.data_access_utils import RDSAccessUtils


In [ ]:
os.environ['PLALI_SQL_CREDENTIALS'] = '/run/secrets/plali_sql_credentials.json'
rds = RDSAccessUtils(json.load(open(os.environ['PLALI_SQL_CREDENTIALS'])))
query = """
    select * from plali.plali_annotations x
    inner join 
    ( select a.id as plali_image_id, a.images, a.metadata, b.id as workflow_id, b.name from plali.plali_images a
    inner join plali.plali_workflows b
    on a.workflow_id = b.id ) y
    on x.plali_image_id = y.plali_image_id
    where workflow_id = '00000000-0000-0000-0000-000000000048';
"""

df = rds.extract_from_database(query)

In [ ]:
df['is_low_turbidity'] = df.metadata.apply(lambda x: 'low_turbidity' in x['tags'])
df['is_medium_turbidity'] = df.metadata.apply(lambda x: 'medium_turbidity' in x['tags'])
df['is_high_turbidity'] = df.metadata.apply(lambda x: 'high_turbidity' in x['tags'])

<h1> Evaluation </h1>

In [ ]:
num_annotated_crops_list = []
num_detected_crops_list = []

mask = df.is_low_turbidity == 1
for idx, row in df[mask].iterrows():
    annotated_crops = row.annotation.get('annotations')
    num_annotated_crops = len([ann for ann in annotated_crops if ann['label'] == 'FULL']) if annotated_crops is not None else 0
    num_detected_crops = len([crop for crop in row.metadata['crops'] if crop['category_id'] == 2])
    num_annotated_crops_list.append(num_annotated_crops)
    num_detected_crops_list.append(num_detected_crops)
    
tdf = pd.DataFrame({'num_annotated_crops': num_annotated_crops_list, 
                    'num_detected_crops': num_detected_crops_list})

<h1> Create video </h1>

<h2> Test helper methods </h2>

In [ ]:
from crop_video_generation import get_image_f, process_detected_crops, process_annotated_crops, overlay_crops_on_image, stitch_frames_into_video

In [ ]:
row = df[mask].iloc[428]
image_s3_url = row.images[0]
detected_crops_original = [crop for crop in row.metadata['crops'] if crop['category_id'] == 2]
annotated_crops_original = [ann for ann in row.annotation['annotations'] if ann['label'] == 'FULL']
detected_crops = process_detected_crops(detected_crops_original)
annotated_crops = process_annotated_crops(annotated_crops_original)

f = get_image_f(image_s3_url)
output_f = overlay_crops_on_image(f, detected_crops, annotated_crops, 'hello')




In [ ]:
output_fs = []
mask = df.is_low_turbidity == 1

count = 0
for idx, row in df[mask].iterrows():
    image_s3_url = row.images[0]
#     detected_crops_original = [crop for crop in row.metadata['crops'] if crop['category_id'] == 2]
#     detected_crops_original = [crop for crop in row.metadata['crops'] if crop['category_id'] == 2]
    if 'annotations' in row.annotation:
        full_crops_original = [ann for ann in row.annotation['annotations'] if ann['label'] == 'FULL']
        partial_crops_original = [ann for ann in row.annotation['annotations'] if ann['label'] == 'PARTIAL']
    else:
        full_crops_original = []
        partial_crops_original = []
        
#     detected_crops = process_detected_crops(detected_crops_original)
#     annotated_crops = process_annotated_crops(annotated_crops_original)
    full_crops = process_annotated_crops(full_crops_original)
    partial_crops = process_annotated_crops(partial_crops_original)

    f = get_image_f(image_s3_url)
    output_f = overlay_crops_on_image(f, full_crops, partial_crops, '{} {}'.format(count, f))
    output_fs.append(output_f)
    
    if count % 10 == 0:
        print(count)

    count += 1


In [ ]:
video_f = '/root/data/alok/biomass_estimation/playground/gopro/video.avi'
stitch_frames_into_video(output_fs, video_f)

In [ ]:
plt.figure(figsize=(20,20))
plt.imshow(cv2.cvtColor(cv2.imread(output_fs[428]), cv2.COLOR_BGR2RGB))

In [ ]:
df[mask].annotation.iloc[438]